# Custom Entity Recognizer

### 1.  Import libraries necessary for the notebook

In [1]:
from datetime import datetime
import boto3
import json

### 2.  Identify your account number

In [2]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
print("Your account id is {}".format(account_id))

Your account id is 348052051973


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### 3.  Create the bucket for the lab (should already exist from Lab1)

In [3]:
bucket_name = "comprehend-labs" + account_id +  "-2"
print ("Bucket name used is " + bucket_name)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

if (s3.Bucket(bucket_name).creation_date is None):
    s3_client.create_bucket(Bucket=bucket_name)
    print ("Created bucket " + bucket_name)
else:
    print ("Bucket Exists")

Bucket name used is comprehend-labs348052051973-2
Bucket Exists


### 4.  Download the training data [entity list, docs], and the test data, then upload to the s3 bucket.

In [4]:
s3_client = boto3.client('s3')
s3_entity_prefix = 'entity-training'
host_name = 'http://d1fjxffqn7wkdo.cloudfront.net'
!wget {host_name}/aws-offerings.csv
response = s3_client.upload_file('./aws-offerings.csv', bucket_name, "{}/aws-offerings.csv".format(s3_entity_prefix))

!wget {host_name}/aws-offerings-docs.txt
response = s3_client.upload_file('./aws-offerings-docs.txt', bucket_name, "{}/aws-offerings-docs.txt".format(s3_entity_prefix))
        
!wget {host_name}/aws-offerings-test.txt
response = s3_client.upload_file('./aws-offerings-test.txt', bucket_name, "{}/aws-offerings-test.txt".format(s3_entity_prefix))

--2022-05-26 03:09:56--  http://d1fjxffqn7wkdo.cloudfront.net/aws-offerings.csv
Resolving d1fjxffqn7wkdo.cloudfront.net (d1fjxffqn7wkdo.cloudfront.net)... 99.84.185.76, 99.84.185.169, 99.84.185.9, ...
Connecting to d1fjxffqn7wkdo.cloudfront.net (d1fjxffqn7wkdo.cloudfront.net)|99.84.185.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17492 (17K) [text/csv]
Saving to: ‘aws-offerings.csv’

aws-offerings.csv   100%[===================>]  17.08K  --.-KB/s    in 0.001s  

2022-05-26 03:09:56 (29.5 MB/s) - ‘aws-offerings.csv’ saved [17492/17492]

--2022-05-26 03:09:56--  http://d1fjxffqn7wkdo.cloudfront.net/aws-offerings-docs.txt
Resolving d1fjxffqn7wkdo.cloudfront.net (d1fjxffqn7wkdo.cloudfront.net)... 99.84.185.49, 99.84.185.76, 99.84.185.169, ...
Connecting to d1fjxffqn7wkdo.cloudfront.net (d1fjxffqn7wkdo.cloudfront.net)|99.84.185.49|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262659 (257K) [text/plain]
Saving to: ‘aws-offerings-docs

### 5.  Let's take a look at the data

In [5]:
!head -20 aws-offerings.csv

"Text", "Type"
"ACM", "AWS_OFFERING"
"ACM PCA", "AWS_OFFERING"
"ACM Private CA", "AWS_OFFERING"
"AD Connector", "AWS_OFFERING"
"AMS", "AWS_OFFERING"
"API Gateway", "AWS_OFFERING"
"AWS", "AWS_OFFERING"
"AWS Amplify", "AWS_OFFERING"
"AWS App Mesh", "AWS_OFFERING"
"AWS AppSync", "AWS_OFFERING"
"AWS Application Auto Scaling", "AWS_OFFERING"
"AWS Application Discovery Service", "AWS_OFFERING"
"AWS Artifact", "AWS_OFFERING"
"AWS Auto Scaling", "AWS_OFFERING"
"AWS Backup", "AWS_OFFERING"
"AWS Batch", "AWS_OFFERING"
"AWS Billing and Cost Management", "AWS_OFFERING"
"AWS Blockchain Templates", "AWS_OFFERING"
"AWS CLI", "AWS_OFFERING"


In [6]:
!head -20 aws-offerings-docs.txt

AWS X-Ray Supports Analytics
Use Analytics in X-Ray and delve into traces to quickly pinpoint issues that may effect your application and its underlying services.
Amazon Sumerian Service Update on 2019-04-26
Service improvements for Amazon Sumerian.
Release: Amazon GameLift on 2019-04-25
Realtime Servers provides ready-to-go, customizable game servers for mobile multiplayer games.
AWS Budgets now Supports Instance Family Filtering for Reservation Coverage Alerts
Starting today, you can use AWS Budgets to create Reservation Coverage budgets to monitor your Amazon EC2 reservations for a given instance family, and receive alerts when your reservation coverage falls below the threshold you define.
Amazon Sumerian Service Update on 2019-04-08
Service improvements for Amazon Sumerian.
AWS X-Ray Supports AWS App Mesh Integration
Use X-Ray to trace communications through AWS App Mesh's service mesh as it networks across multiple types of computer infrastructure.
AWS X-Ray Groups: Deep Dive Dev

### 6.  Keep these outputs for the manual steps you're about to do. You can copy the outputs to a text doc locally (e.g., your laptop)

In [7]:
print("Entity List Location:  s3://{}/{}/aws-offerings.csv".format(bucket_name,s3_entity_prefix))
print("Training Documents Location:  s3://{}/{}/aws-offerings-docs.txt".format(bucket_name,s3_entity_prefix))
print("Test Documents Location:  s3://{}/{}/aws-offerings-test.txt".format(bucket_name,s3_entity_prefix))
print("Bucket Path:  s3://{}".format(bucket_name))

Entity List Location:  s3://comprehend-labs348052051973-2/entity-training/aws-offerings.csv
Training Documents Location:  s3://comprehend-labs348052051973-2/entity-training/aws-offerings-docs.txt
Test Documents Location:  s3://comprehend-labs348052051973-2/entity-training/aws-offerings-test.txt
Bucket Path:  s3://comprehend-labs348052051973-2


# Now let's go back to the console and kick off the jobs manually

---

## For extra credit, here are the steps to continue doing this in code

### 1. Add IAM permissions to SageMaker
For SageMaker to kick off trainig jobs, it needs the ability to pass a role to the Comprehend service.  
In the IAM console, add the following policy to the role that the SageMaker notebook created.

### 2. Get the ARN for the role we created in the first Lab

In [ ]:
#This name should match the name of the role that was created in the first lab.
role_name_base = 'AmazonComprehendServiceRoleS3FullAccess-ComprehendLabs'
prefix_random_numbers = '' #If you added random numbers to the end of the 'ComprehendLabs' prefix, put them here
if not prefix_random_numbers:
    role_name = "{}{}".format(role_name_base,prefix_random_numbers)
else:
    role_name = role_name_base
iam_client = boto3.client("iam")
response = iam_client.get_role(
    RoleName=role_name
)
comprehend_arn = response['Role']['Arn']
print("The ARN for the role is {}".format(comprehend_arn))

### 3. Start training job

In [ ]:
comprehend_client = boto3.client("comprehend")
response = comprehend_client.create_entity_recognizer(
    RecognizerName="Recognizer-Name-Goes-Here-{}".format(datetime.now()).replace(' ','-').replace(':','-').replace('.','-'),
    LanguageCode="en",
    DataAccessRoleArn=comprehend_arn,
    InputDataConfig={
        "EntityTypes": [
            {
                'Type': "AWS_OFFERING"
            }
        ],
        'EntityList': {
            'S3Uri': "s3://{}/{}/aws-offerings.csv".format(bucket_name,s3_entity_prefix)
        },
        'Documents': {
            'S3Uri': "s3://{}/{}/aws-offerings-docs.txt".format(bucket_name,s3_entity_prefix)
        },
        
    }
)
recognizer_arn = response["EntityRecognizerArn"]
print("The ARN for the entity recognizer is {}".format(recognizer_arn))

### 4.  Check the status of the training job

In [ ]:
response = comprehend_client.describe_entity_recognizer(
    EntityRecognizerArn=recognizer_arn
)
#The possible statuses for the custom entity recognizer are: 'SUBMITTED'|'TRAINING'|'DELETING'|'STOP_REQUESTED'|'STOPPED'|'IN_ERROR'|'TRAINED'
print("The status of the custom entity recognizer is {}".format(response['EntityRecognizerProperties']['Status']))

### 5.  Lets look at how the training did

In [ ]:
response = comprehend_client.describe_entity_recognizer(
    EntityRecognizerArn=recognizer_arn
)
if response['EntityRecognizerProperties']['Status'] == 'TRAINED':
    print(json.dumps(response['EntityRecognizerProperties']['RecognizerMetadata'], indent=2))
else:
    print ("Training job has not completed yet.  Please wait to check training performance until it has.")

### 6.  Start a batch entity recognition job

In [ ]:
response = comprehend_client.describe_entity_recognizer(
    EntityRecognizerArn=recognizer_arn
)
if response['EntityRecognizerProperties']['Status'] == 'TRAINED':
    response = comprehend_client.start_entities_detection_job(
        JobName='AWS_OFFERING-001',
        EntityRecognizerArn=recognizer_arn,
        LanguageCode="en",
        DataAccessRoleArn=comprehend_arn,
        InputDataConfig={
            'S3Uri': "s3://{}/{}/aws-offerings-test.txt".format(bucket_name,s3_entity_prefix),
            'InputFormat': 'ONE_DOC_PER_LINE'
        },
        OutputDataConfig={
            'S3Uri': "s3://{}/{}/results/".format(bucket_name,s3_entity_prefix)
        }
    )
    job_id = response['JobId']
else:
    print ("Training job has not completed yet.  Please wait to start batch entity recognitino job until it has.")

### 7.  Check the status of the bacth transform job

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
print("The status of the batch entity detection job is {}".format(response['EntitiesDetectionJobProperties']['JobStatus']))

### 8.  Download the output of the batch job

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
if response['EntitiesDetectionJobProperties']['JobStatus'] == "COMPLETED":
    output_s3_uri = response['EntitiesDetectionJobProperties']['OutputDataConfig']['S3Uri']
    s3_key = output_s3_uri.replace("s3://{}/".format(bucket_name),'')
    s3.meta.client.download_file(bucket_name, s3_key, 'output.tar.gz')
    !tar zxvf output.tar.gz
else:
    print("Batch transformation job not complete.  Please wait until this job is completed before attempting to view output.")

### 9.  Let's review the test data and the output

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
if response['EntitiesDetectionJobProperties']['JobStatus'] == "COMPLETED":
    !head -20 aws-offerings-test.txt
else:
    print("Batch transformation job not complete.  Please wait until this job is completed before attempting to view output.")

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
if response['EntitiesDetectionJobProperties']['JobStatus'] == "COMPLETED":
    !cat output
else:
    print("Batch transformation job not complete.  Please wait until this job is completed before attempting to view output.")